In [2]:
import pandas as pd
import zipfile
import plotly.graph_objects as go


In [3]:
with zipfile.ZipFile('dataframes.zip', 'r') as zipf:
    csv_files = [name for name in zipf.namelist() if name.endswith('.csv')]
    dataframes = [pd.read_csv(zipf.open(csv_file)) for csv_file in csv_files]

In [33]:
stations_sample = [pd.read_csv('sample_station_grid.csv')]

In [ ]:
from mpl_toolkits.basemap import Basemap
m = Basemap()

def is_land(latitude, longitude):
    x, y = m(longitude, latitude)
    if (-100 < x < -74) and (40 < y < 55): # plot great lakes
        return True
    return m.is_land(x, y)

melteds = []

for i, df in enumerate(dataframes):
    df_melted = df.melt(id_vars=['Unnamed: 0'], var_name='Longitude', value_name='Temperature')
    df_melted.columns = ['Latitude', 'Longitude', 'Temperature']
    df_melted.dropna(inplace=True)
    df_melted['Latitude'] = df_melted['Latitude'].astype(float)
    df_melted['Longitude'] = df_melted['Longitude'].astype(float)
    df_melted['Temperature'] = df_melted['Temperature'].astype(float)
    df_melted = df_melted[df_melted.apply(lambda row: is_land(row['Latitude'], row['Longitude']), axis=1)]
    melteds.append(df_melted)

In [40]:
def create_frame(day_index, df):

    scatter = go.Scattergeo(
        lat=df['Latitude'],
        lon=df['Longitude'],
        text=df['Temperature'],
        mode='markers',
        marker=dict(
            size=5,
            color=df['Temperature'],
            colorscale='thermal',
            cmin = -30,
            cmax = 40,
            showscale=True,
            colorbar=dict(title="Temperature")
        )
    )
    
    frame = go.Frame(data=[scatter], name=str(day_index))
    return frame

In [41]:
fig = go.Figure()

frames = [create_frame(i, df) for i, df in enumerate(melteds)]
fig.frames = frames

if frames:
    fig.add_trace(frames[0].data[0])

sliders = [{
    'steps': [{'method': 'animate', 'args': [[frame.name], {'frame': {'duration': 20, 'redraw': True}, 'mode': 'immediate'}], 'label': str(i)} for i, frame in enumerate(fig.frames)],
    'transition': {'duration': 20},
    'x': 0.1,
    'len': 0.9,
    'xanchor': 'left',
    'y': 0,
    'yanchor': 'top'
}]

fig.update_layout(
    sliders=sliders,
    geo=dict(
        projection_scale=2.5,
        center=dict(lat=42.50, lon=-98.35),
        showocean=True,
        showland=True,
        landcolor="rgb(229, 229, 229)",
        oceancolor="rgb(204, 204, 255)",
        lakecolor="rgb(204, 204, 255)",
        scope='north america',
    ),
    autosize=False,
    width=1300,
    height=950,
    updatemenus=[{
        'type': 'buttons',
        'showactive': False,
        'y': 0.8,
        'x': 0.0,
        'xanchor': 'right',
        'yanchor': 'top',
        'pad': {'t': 60, 'r': 10},
        'buttons': [
            {
                'label': '▶',
                'method': 'animate',
                'args': [None, {
                    'frame': {'duration': 20, 'redraw': True},
                    'fromcurrent': True,
                    'transition': {'duration': 10, 'easing': 'quadratic-in-out'}
                }]
            },
            {
                'label': '⏸',
                'method': 'animate',
                'args': [[None], {
                    'mode': 'immediate',
                    'frame': {'duration': 0, 'redraw': False},
                }]
            },
            {
                'label': '⏮',
                'method': 'animate',
                'args': [[frames[0].name], {
                    'frame': {'duration': 0, 'redraw': True},
                    'mode': 'immediate',
                    'transition': {'duration': 0}
                }]
            }
        ]
    }]
)



# Show figure
fig.show()

: 